# 东京奥运会数据可视化

## 东京奥运会每日奖牌榜动态展示

In [1]:
import requests
import pandas as pd
import re
from pandas import DataFrame
from pyecharts.charts import *
from pyecharts import options as opts
from pyecharts.commons.utils import JsCode
from datetime import datetime

In [2]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [3]:
res = requests.get('https://app-sc.miguvideo.com/vms-livedata/olympic-medal/detail-total/15/110000004609')
data = res.json()

medal_data = data['body']['medalTableDetail']

df = pd.DataFrame(medal_data)
df

,date,awardTime,medalType,sportsName,countryId,countryName,bigItemId,bigItemName,minorItemId,minorItemName,isBreakWorldRecord,isBreakOlyRecord,mgdbId,programId
0,20210808,2021-08-08 16:44:13,1,塞尔维亚男子水球队,107,SRB,36414128,水球,36415660,男子组,0,0,NaN,NaN
1,20210808,2021-08-08 16:44:13,2,希腊男子水球队,171,GRE,36414128,水球,36415660,男子组,0,0,NaN,NaN
2,20210808,2021-08-08 15:34:46,1,法国女子手球队,144,FRA,36414130,手球,36415705,女子组,0,0,NaN,NaN
3,20210808,2021-08-08 15:34:46,2,俄奥委会女子手球队,23062,ROC,36414130,手球,36415705,女子组,0,0,NaN,NaN
4,20210808,2021-08-08 14:49:07,1,巴霍迪尔·贾洛洛夫,15,UZB,36414129,拳击,36415686,男子超重量级(91公斤以上),0,0,NaN,NaN
5,20210808,2021-08-08 14:49:07,2,理查德·托雷斯,5,USA,36414129,拳击,36415686,男子超重量级(91公斤以上),0,0,NaN,NaN
6,20210808,2021-08-08 14:13:08,1,劳伦·普莱斯,211,GBR,36414129,拳击,36415701,女子中量级(69-75公斤),0,0,NaN,NaN
7,20210808,2021-08-08 14:10:00,2,李倩,26,CHN,36414129,拳击,36415701,女子中量级(69-75公斤),0,0,120000178076,NaN
8,20210808,2021-08-08 14:07:59,1,美国女子排球队,5,USA,36414127,排球,36415658,女子组,0,0,NaN,NaN
9,20210808,2021-08-08 14:07:59,2,巴西女子排球队,16,BRA,36414127,排球,36415658,女子组,0,0,NaN,NaN


In [4]:
df2= df.groupby(['countryName','countryId','date'],as_index=False).agg({'medalType':'count'})
#数据透视表
df_medal=pd.pivot_table(df2,values='medalType',index=[u'countryName'],columns='date',fill_value=0)
df_medal= df_medal.cumsum(axis=1)
df_medal= df_medal.reset_index()
df_medal

date,countryName,20210724,20210725,20210726,20210727,20210728,20210729,20210730,20210731,20210801,20210802,20210803,20210804,20210805,20210806,20210807,20210808
0,ARG,0,0,0,0,1,1,1,1,1,1,1,1,1,2,3,3
1,ARM,0,0,0,0,0,0,0,0,0,1,3,3,3,4,4,4
2,AUS,0,3,6,9,16,20,22,27,31,33,33,36,41,44,46,46
3,AUT,0,1,1,2,3,3,4,5,5,5,5,5,7,7,7,7
4,AZE,0,0,0,0,0,0,1,1,2,2,3,3,3,4,7,7
5,BAH,0,0,0,0,0,0,0,0,0,0,0,0,1,2,2,2
6,BEL,1,1,1,2,2,2,2,2,3,3,3,3,5,5,6,7
7,BER,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1
8,BLR,0,0,0,0,0,0,0,1,2,2,2,2,3,5,7,7
9,BOT,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1


In [5]:
range_color=[
                '#313695',
                '#4575b4',
                '#74add1',
                '#abd9e9',
                '#e0f3f8',
                '#ffffbf',
                '#fee090',
                '#fdae61',
                '#f46d43',
                '#d73027',
                '#a50026'
            ]

In [6]:
range_date= df_medal.columns.tolist()
range_date.remove('countryName')

In [7]:
def chart_(x_data, y_data, data_pair, date):
    bar = Bar()
    bar.add_xaxis(
        x_data[::-1]
    )
    bar.add_yaxis(
        '',
        y_data[::-1],
        category_gap='40%',
        label_opts=opts.LabelOpts(
            is_show=True,
            formatter='{c} 块奖牌',
            position='insideLeft'),
    )
    bar.set_global_opts(
        xaxis_opts=opts.AxisOpts(is_show=False),
        yaxis_opts=opts.AxisOpts(
            axisline_opts=opts.AxisLineOpts(is_show=False),
            axistick_opts=opts.AxisTickOpts(is_show=False)),
        title_opts=opts.TitleOpts(
            title='2020东京奥运会奖牌榜',
            pos_left='2%',
            pos_top='2%',
            title_textstyle_opts=opts.TextStyleOpts(
                color='#00BFFF', font_size=20)
        ),
        visualmap_opts=opts.VisualMapOpts(
            is_show=False,
            max_=20,
            series_index=0,
            range_color=range_color
        ),
        graphic_opts=[
            opts.GraphicGroup(graphic_item=opts.GraphicItem(
                bounding="raw",
                right=400,
                bottom=100,
                z=100),
            # 文字显示区域配置
            children=[
                # 文本设置
                opts.GraphicText(
                    graphic_item=opts.GraphicItem(
                        left="center", top="center", z=100
                    ),
                    graphic_textstyle_opts=opts.GraphicTextStyleOpts(
                        # 要显示的文本
                        text='{}'.format(date),
                        font="bold 86px Microsoft YaHei",
                        graphic_basicstyle_opts=opts.GraphicBasicStyleOpts(
                            fill="rgba(0,0,0,.2)"
                        ),
                    ),
                ),
            ],
        )
        ]
    )
    bar.reversal_axis()

    pie = Pie()
    pie.add(
        "",
        data_pair,
        # 指定饼图中心位置
        center=["70%", "60%"],
        # 将饼图尺寸相应缩小，不然会重叠
        radius=["25%", "35%"],
    )

    pie.set_global_opts(
        legend_opts=opts.LegendOpts(is_show=False),
    )
    pie.set_colors(range_color)


    grid = Grid()

    grid.add(bar, is_control_axis_index=False, grid_opts=opts.GridOpts(pos_left='10%', pos_right='5%', pos_top='5%', pos_bottom='5%'))
    grid.add(pie, is_control_axis_index=True, grid_opts=opts.GridOpts(pos_left='5%', pos_right='5%', pos_top='5%', pos_bottom='5%'))
    return grid

In [8]:
timeline = Timeline(init_opts=opts.InitOpts(theme='light', width='980px', height='800px'))
timeline.add_schema(
    is_auto_play=True,  # 自动播放
    is_loop_play=True,  # 循环播放
    is_timeline_show=False
)


for date in range_date:
    data_pair, x_data, y_data = [], [], []
    df_medal.sort_values(by=date, ascending=False, inplace=True)
    df_medal.reset_index(drop=True, inplace=True)
    for idx, row in df_medal.iterrows():
        if row[date] > 0:
            if idx < 10:
                x_data.append(row['countryName'])
                if row['countryName'] == 'CHN':
                    y_data.append(
                        opts.BarItem(
                            name=row['countryName'],
                            value=row[date],
                            itemstyle_opts={
                                'normal': {
                                    # 阴影颜色
                                    'shadowColor': 'rgba(255, 0, 0, 0.5)',
                                    'shadowBlur': 20,  # 阴影大小
                                    'shadowOffsetY': 5,  # Y轴方向阴影偏移
                                    'shadowOffsetX': 5,  # x轴方向阴影偏移
                                    'borderColor': 'rgba(255, 0, 0, 1)',
                                    'borderWidth': 3
                                }
                            }
                        )
                    )
                else:
                    y_data.append(
                        opts.BarItem(
                            name=row['countryName'],
                            value=row[date],
                            itemstyle_opts={
                                'normal': {
                                    # 阴影颜色
                                    'shadowColor': 'rgba(0, 0, 0, .5)',
                                    'shadowBlur': 5,  # 阴影大小
                                    'shadowOffsetY': 2,  # Y轴方向阴影偏移
                                    'shadowOffsetX': 2,  # x轴方向阴影偏移
                                    'borderColor': '#fff'
                                }
                            }
                        )
                    )
                if idx < 5:
                    data_pair.append(
                        opts.PieItem(
                            name=row['countryName'],
                            value=row[date],
                            label_opts=opts.LabelOpts(
                                is_show=True, formatter='{b}\n{d}%')
                        )
                    )
                else:
                    data_pair.append(
                        opts.PieItem(
                            name=row['countryName'],
                            value=row[date],
                            label_opts=opts.LabelOpts(
                                is_show=False, position='inside')
                        )
                    )
            else:
                data_pair.append(
                    opts.PieItem(
                        name=row['countryName'],
                        value=row[date],
                        label_opts=opts.LabelOpts(
                            is_show=False, position='inside')
                    )
                )

    c = chart_(x_data, y_data, data_pair, date)
    timeline.add(c, '{}'.format(date))
    
timeline.render_notebook()

# timeline.render('olympic.html')

## 2020东京奥运会创全新世界纪录项目展示

In [9]:
df3= df.loc[df['isBreakWorldRecord']==1]

In [10]:
df_symbol=DataFrame({'bigItemName':['举重','场地自行车赛','田径','射击','游泳'],
            'svg':['https://olympics.com/tokyo-2020/en/d3images/pictograms/olympics/picto-wlf.svg',
                   'https://olympics.com/tokyo-2020/en/d3images/pictograms/olympics/picto-ctr.svg',
                  'https://olympics.com/tokyo-2020/en/d3images/pictograms/olympics/picto-ath.svg',
                  'https://olympics.com/tokyo-2020/en/d3images/pictograms/olympics/picto-sho.svg',
                  'https://olympics.com/tokyo-2020/en/d3images/pictograms/olympics/picto-ows.svg']})

In [11]:
df_worldrecord= df3.merge(df_symbol, on ='bigItemName')
df_worldrecord= df_worldrecord.sort_values(by='date')
df_worldrecord.reset_index(drop=True)

,date,awardTime,medalType,sportsName,countryId,countryName,bigItemId,bigItemName,minorItemId,minorItemName,isBreakWorldRecord,isBreakOlyRecord,mgdbId,programId,svg
0,20210725,2021-07-25 00:00:00,1,莫莉·奥卡拉汉等,157,AUS,36414095,游泳,36415066,女子4x100米自由泳接力,1,1,NaN,NaN,https://olympics.com/tokyo-2020/en/d3images/pi...
1,20210728,2021-07-28 20:33:00,1,石智勇,26,CHN,36414108,举重,36415350,男子73公斤级,1,1,120000167927,NaN,https://olympics.com/tokyo-2020/en/d3images/pi...
2,20210729,2021-07-29 11:54:00,1,汤慕涵/杨浚瑄/张雨霏/李冰洁/张一璠/董洁,26,CHN,36414095,游泳,36415068,女子4X200米自由泳接力,1,1,120000176421,NaN,https://olympics.com/tokyo-2020/en/d3images/pi...
3,20210730,2021-07-30 09:50:31,1,塔佳娜·舍恩梅克,178,RSA,36414095,游泳,36415056,女子200米蛙泳,1,1,NaN,NaN,https://olympics.com/tokyo-2020/en/d3images/pi...
4,20210731,2021-07-31 10:56:12,1,凯瑟琳·道森等,211,GBR,36414095,游泳,36415072,男女混合4x100米混合泳接力,1,1,NaN,NaN,https://olympics.com/tokyo-2020/en/d3images/pi...
5,20210731,2021-07-31 09:37:13,1,塞勒博·德雷赛尔,5,USA,36414095,游泳,36415018,男子100米蝶泳,1,1,NaN,NaN,https://olympics.com/tokyo-2020/en/d3images/pi...
6,20210801,2021-08-01 20:40:12,1,尤利玛·罗哈斯,162,VEN,36414097,田径,36415178,女子三级跳远,1,1,NaN,NaN,https://olympics.com/tokyo-2020/en/d3images/pi...
7,20210801,2021-08-01 10:56:44,1,约瑟夫·阿姆斯特朗等,5,USA,36414095,游泳,36415035,男子4x100米混合泳接力,1,1,NaN,NaN,https://olympics.com/tokyo-2020/en/d3images/pi...
8,20210802,2021-08-02 17:12:00,1,鲍珊菊/钟天使,26,CHN,36414099,场地自行车赛,36415232,女子团体争先赛,1,1,120000176629,NaN,https://olympics.com/tokyo-2020/en/d3images/pi...
9,20210802,2021-08-02 16:43:00,1,张常鸿,26,CHN,36414120,射击,36415532,男子50米步枪三姿,1,1,120000168274,NaN,https://olympics.com/tokyo-2020/en/d3images/pi...


In [12]:
y_data1 = []
counter = 0
position = ['left', 'right']
for idx, row in df_worldrecord.iterrows():
    #formatter参数支持字符串模板和回调函数两种形式，其中字符串模板显示有四个选项：{a}（系列名称），{b}（数据名称），{c}（数值数组）, {d}（无）
    msg = '{bbb|%s}\n{aaa|%s}\n{bbb|%s/%s}' % (row['awardTime'], row['sportsName'], row['bigItemName'], row['minorItemName'])
    # 单个数据项配置
    l_item = opts.LineItem(
        name=df_worldrecord.shape[0],
        value=counter,
        symbol='image://%s'%(row['svg']),#标记图形的类型
        symbol_keep_aspect= False,
        symbol_size=30,
        label_opts=opts.LabelOpts(
            is_show=True, 
            font_size=16,
            position=position[counter%2], 
            formatter=msg,
            rich = {
                'aaa': {
                    'fontSize': 18, 
                    'color': '#8b1f30', 
                    'fontWeight':'bold', 
                    'align':position[(counter+1)%2],
                    },
                'bbb': {
                    'fontSize': 15, 
                    'color': '#000', 
                    'align':position[(counter+1)%2]}}
            )
    )
    y_data1.append(l_item)
    counter+=1

In [13]:
line = Line(
    init_opts=opts.InitOpts(
        theme='light',
        width='1000px',
        height='1200px',
        bg_color='white'
    )
)
line.add_xaxis(
    ['']
)
line.add_yaxis(
    '',
    y_data1,
    linestyle_opts={
        'normal': {
            'width': 4,  # 设置线宽
            'color':'#8b1f30',

        }
    },

    tooltip_opts=opts.TooltipOpts(is_show=False)
)

line.set_global_opts(
    xaxis_opts=opts.AxisOpts(is_show=False, type_='category'),
    yaxis_opts=opts.AxisOpts(is_show=False, type_='value', max_=len(y_data1)),
    title_opts=opts.TitleOpts(
        title="2020东京奥运会全新世界纪录", pos_left='center', pos_top='2%',
        title_textstyle_opts=opts.TextStyleOpts(color='#8b1f30', font_size=24)
    ),
    graphic_opts=[
                opts.GraphicGroup(
                            graphic_item=opts.GraphicItem(id_='1',left="center", top="center", z=-1),
                            children=[# tokyo 
                                    opts.GraphicImage(graphic_item=opts.GraphicItem(id_="logo",
                                                                                    left='center',
                                                                                    z=-3),
                                                      graphic_imagestyle_opts=opts.GraphicImageStyleOpts(
                                        image="https://olympics.com/tokyo-2020/en/d3images/emblem/olympics/emblem-tokyo2020.svg",
                                        width=800,
                                        height=1000,
                                        opacity=0.1,)
                                    )
                                ]
                                )
                                ]
)


line.render_notebook()